<a href="https://colab.research.google.com/github/vohoaidanh/AIGCDetectBenchmark/blob/main/colab/AIGCDetectBenchmark_train_val.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AIGCDetectBenchmark


In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!pip install ftfy
!pip install natsort
!pip install tensorboardX
!pip install blobfile
!pip install mpi4py


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 11.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.5-cp310-cp310-linux_x86_64.whl size=2746512 sha256=4a199717c1044e2ae8f8597f581ccbfbd72e3969adb07f66223118a628cce118
  Stored in directory: /root/.cache/pip/wheels/18/2b/7f/c852523089e9182b45fca50ff56f49a51eeb6284fd25a66713
Successfully built mpi4py


In [2]:
!git clone https://github.com/vohoaidanh/AIGCDetectBenchmark.git

Cloning into 'AIGCDetectBenchmark'...
remote: Enumerating objects: 618, done.
remote: Counting objects: 100% (618/618), done.
remote: Compressing objects: 100% (439/439), done.
remote: Total 618 (delta 252), reused 494 (delta 160), pack-reused 0
Receiving objects: 100% (618/618), 7.18 MiB | 16.68 MiB/s, done.
Resolving deltas: 100% (252/252), done.


In [4]:
!cp /content/drive/MyDrive/DATASETS/real_gen_dataset.zip -d /content
!cp /content/drive/MyDrive/DATASETS/RealFakeDB_tiny_test.zip -d /content

In [ ]:
!unzip /content/real_gen_dataset.zip -d /content/real_gen_dataset

In [ ]:
!unzip /content/RealFakeDB_tiny_test.zip -d /content/dataset

In [8]:
!rm -r /content/real_gen_dataset/real_gen_dataset/test

In [10]:
%cd /content/AIGCDetectBenchmark

/content/AIGCDetectBenchmark


In [ ]:
!python train.py --name test_with_origin --dataroot /content/AIGCDetectBenchmark/dataset \
--checkpoints_dir /content/drive/MyDrive/WEIGHTS/Intrinsic_detector \
--detect_method CNNSpot --blur_prob 0.1 --blur_sig 0.0,3.0 --jpg_prob 0.1 --jpg_method cv2,pil --jpg_qual 30,100

----------------- Options ---------------
                 CropSize: 224                           
               batch_size: 64                            
                    beta1: 0.9                           
                blur_prob: 0.1                           
                 blur_sig: 0.0,3.0                       
          checkpoints_dir: /content/drive/MyDrive/WEIGHTS/Intrinsic_detector	[default: ./checkpoints]
                  classes: ['airplane', 'bird', 'bicycle', 'boat', 'bottle', 'bus', 'car', 'cat', 'cow', 'chair', 'diningtable', 'dog', 'person', 'pottedplant', 'motorbike', 'tvmonitor', 'train', 'sheep', 'sofa', 'horse']	[default: airplane,bird,bicycle,boat,bottle,bus,car,cat,cow,chair,diningtable,dog,person,pottedplant,motorbike,tvmonitor,train,sheep,sofa,horse]
           continue_train: False                         
                 data_aug: False                         
                 dataroot: /content/AIGCDetectBenchmark/dataset	[default: /hotdata/

In [ ]:
!cp /content/drive/MyDrive/WEIGHTS/Intrinsic_detector/test_with_intrinsic/model_epoch_best.pth /content/AIGCDetectBenchmark/checkpoints

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

# Đường dẫn đến event file được tạo bởi SummaryWriter
event_file_path = 'checkpoints/test_with_PIE_Net/val/events.out.tfevents.1709110494.4cc1337ca71b'

# Đọc dữ liệu từ event file
summary_iterator = tf.compat.v1.train.summary_iterator(event_file_path)

# Khởi tạo các list để lưu giữ dữ liệu
steps = []
train_losses = []
train_accuracies = []

# Lặp qua từng event trong event file
for event in summary_iterator:
    if event.HasField('summary'):
        for value in event.summary.value:
            if value.tag == 'accuracy':
                steps.append(event.step)
                train_losses.append(value.simple_value)
            elif value.tag == 'ap':
                train_accuracies.append(value.simple_value)

# Vẽ đồ thị train loss
plt.plot(steps, train_losses, label='accuracy')
plt.xlabel('Steps')
plt.ylabel('accuracy')
plt.title('Validate accuracy over Steps')
plt.legend()
plt.show()

# Vẽ đồ thị train accuracy
plt.plot(steps, train_accuracies, label='Val AP')
plt.xlabel('Steps')
plt.ylabel('AP')
plt.title('Validate AP over Steps')
plt.legend()
plt.show()


In [ ]:
!python eval_all.py --detect_method intrinsic --model_path /content/AIGCDetectBenchmark/checkpoints/model_epoch_best.pth

dataset
----------------- Options ---------------
                 CropSize: 224                           
           DIRE_modelpath: /data_19/AIGCDetect/AIGCDetect/weights/preprocessing/lsun_bedroom.pt
          LGrad_modelpath: ./weights/preprocessing/karras2019stylegan-bedrooms-256x256_discriminator.pth
            LNP_modelpath: ./weights/preprocessing/sidd_rgb.pth
               batch_size: 64                            
                 blur_sig: 1.0                           
             dataset_name:                               
            detect_method: intrinsic                     	[default: CNNSpot]
                  isTrain: False                         	[default: None]
                    isVal: False                         	[default: None]
               jpg_method: pil                           
                 jpg_qual: 95                            
                 loadSize: 256                           
               model_path: /content/AIGCDetectBenchmar